<a href="https://colab.research.google.com/github/NatSama2/Bootcamp-Analisis-de-Datos/blob/main/Modulo-5/Regresi%C3%B3n_Puntaje_Horas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Regresión Lineal Simple MCO

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot, shapiro, jarque_bera, normaltest, anderson, kstest
from statsmodels.stats.diagnostic import het_breuschpagan, het_white,lilliefors
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import GLS, GLSAR


In [ ]:
data = pd.read_excel("/content/sample_data/Data_Regresion.xlsx", sheet_name='data')
data.head()

In [ ]:
# Este Script sera modificado para tu conveniencia

#1 Definir variables dependiente (Y) e independiente (X)

X = data['Horas(x)']
y= data['Puntaje(y)']

# Añadir una constante a las características para el modelo statsmodels
X_const = sm.add_constant(X)

print("Datos Cargados:")
print(data)
print("-" * 50)

# 2. Construcción del Modelo de Regresión Lineal ---
# Usamos statsmodels para tener un resumen detallado y acceso a los residuos para los supuestos.
model = sm.OLS(y, X_const)
## results = model.fit(cov_type='HAC', cov_kwds={'maxlags':1})
results = model.fit()
print("Resumen del Modelo de Regresión Lineal:")
print(results.summary())
print("-" * 50)

# Obtener los residuos del modelo
residuals = results.resid
fitted_values = results.fittedvalues

# --- 3. Evaluación de Supuestos de la Regresión Lineal ---

## Supuesto 1: Linealidad 📈
# Se asume que la relación entre las variables es lineal.
# Se puede verificar con un gráfico de dispersión de la variable predictora contra la variable objetivo,
# y también con un gráfico de residuos vs. valores predichos.

plt.figure(figsize=(12, 6))

# Gráfico de dispersión de X vs. Y
plt.subplot(1, 2, 1)
sns.scatterplot(x=X, y=y)
plt.title('Linealidad: X vs. Y')
plt.xlabel('X')
plt.ylabel('Y')

# Residuos vs. Valores Predichos: La mejor manera de verificar la linealidad y homocedasticidad
plt.subplot(1, 2, 2)
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos vs. Valores Predichos')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuos')
plt.tight_layout()
plt.show()

print("\n--- Evaluación del Supuesto de Linealidad ---")
print("Observar el gráfico 'X vs. Y': si los puntos siguen una tendencia lineal, el supuesto se cumple.")
print("En el gráfico de 'Residuos vs. Valores Predichos', los residuos deben estar distribuidos aleatoriamente alrededor de cero, sin patrones evidentes (forma de embudo, curva, etc.).")
print("-" * 50)

## Supuesto 2: Normalidad de los Residuos 📊
# Los residuos deben seguir una distribución normal.
# Se puede verificar con gráficos (histograma, Q-Q plot) y pruebas estadísticas.

plt.figure(figsize=(12, 5))

# Histograma de los residuos
plt.subplot(1, 2, 1)
sns.histplot(residuals, kde=True)
plt.title('Histograma de los Residuos')
plt.xlabel('Residuos')
plt.ylabel('Frecuencia')

# Q-Q plot de los residuos
plt.subplot(1, 2, 2)
probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot de los Residuos')
plt.tight_layout()
plt.show()

print("\n--- Evaluación del Supuesto de Normalidad de los Residuos ---")
print("Pruebas estadísticas:")

# Prueba de Shapiro-Wilk (para n < 50)
shapiro_test = shapiro(residuals)
print(f"Shapiro-Wilk Test: Estadístico={shapiro_test.statistic:.3f}, p-value={shapiro_test.pvalue:.3f}")
if shapiro_test.pvalue > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

# Prueba de Jarque-Bera (para n > 2000, pero también útil para n pequeñas)
jarque_bera_test = jarque_bera(residuals)
print(f"Jarque-Bera Test: Estadístico={jarque_bera_test.statistic:.3f}, p-value={jarque_bera_test.pvalue:.3f}")
if jarque_bera_test.pvalue > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")



# Prueba de Agostino-Pearson (D'Agostino's K-squared test)
agostino_test = normaltest(residuals)
print(f"\nAgostino-Pearson Test: Estadístico={agostino_test.statistic:.3f}, p-value={agostino_test.pvalue:.3f}")
if agostino_test.pvalue > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

# Prueba de Anderson-Darling
anderson_test = anderson(residuals, dist='norm')
print(f"\nAnderson-Darling Test: Estadístico={anderson_test.statistic:.3f}")
print("Valores críticos y niveles de significancia:")
for i in range(len(anderson_test.critical_values)):
    sl, cv = anderson_test.significance_level[i], anderson_test.critical_values[i]
    if anderson_test.statistic < cv:
        print(f"  -> Para nivel de significancia {sl}%: estadístico < {cv:.3f} (No rechazar normalidad)")
    else:
        print(f"  -> Para nivel de significancia {sl}%: estadístico >= {cv:.3f} (Rechazar normalidad)")

# Prueba de Kolmogorov-Smirnov (con parámetros estimados)
ks_test = kstest(residuals, 'norm', args=(residuals.mean(), residuals.std()))
print(f"\nKolmogorov-Smirnov Test: Estadístico={ks_test.statistic:.3f}, p-value={ks_test.pvalue:.3f}")
if ks_test.pvalue > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

# Prueba de Lilliefors (Kolmogorov-Smirnov modificada para normalidad con parámetros estimados)
lillie_test = lilliefors(residuals, dist='norm')
print(f"\nLilliefors Test: Estadístico={lillie_test[0]:.3f}, p-value={lillie_test[1]:.3f}")
if lillie_test[1] > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")



print("\nObservar el Q-Q Plot: los puntos deben seguir aproximadamente la línea recta. Las desviaciones indican no normalidad.")
print("-" * 50)

## Supuesto 3: Homocedasticidad (Varianza Constante de los Residuos) 📏
# La varianza de los residuos debe ser constante en todos los niveles de las variables predictoras.
# Se verifica principalmente con el gráfico de residuos vs. valores predichos y pruebas estadísticas.

print("\n--- Evaluación del Supuesto de Homocedasticidad ---")
# El gráfico de residuos vs. valores predichos ya se mostró en la sección de linealidad.
# Buscar patrones en forma de embudo o cono, que indicarían heterocedasticidad.

print("Pruebas estadísticas:")

# Prueba de Breusch-Pagan
bp_test = het_breuschpagan(residuals, X_const)
print(f"Breusch-Pagan Test: LM Statistic={bp_test[0]:.3f}, p-value={bp_test[1]:.3f}, F-statistic={bp_test[2]:.3f}, F-p-value={bp_test[3]:.3f}")
if bp_test[1] > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: hay homocedasticidad (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: hay heterocedasticidad (p <= 0.05).")

# Prueba de White
white_test = het_white(residuals, X_const)
print(f"White Test: LM Statistic={white_test[0]:.3f}, p-value={white_test[1]:.3f}, F-statistic={white_test[2]:.3f}, F-p-value={white_test[3]:.3f}")
if white_test[1] > 0.05:
    print("  -> **No se puede rechazar la hipótesis nula**: hay homocedasticidad (p > 0.05).")
else:
    print("  -> **Se rechaza la hipótesis nula**: hay heterocedasticidad (p <= 0.05).")
print("-" * 50)

## Supuesto 4: Ausencia de Multicolinealidad 🔄
# Las variables predictoras no deben estar altamente correlacionadas entre sí.
# Para este conjunto de datos, solo hay una variable predictora (X), por lo que la multicolinealidad no es un problema.
# Sin embargo, se incluye la sección para demostrar cómo se haría con múltiples variables.

print("\n--- Evaluación del Supuesto de Ausencia de Multicolinealidad ---")
if X.ndim > 1 and X.shape[1] > 1: # Solo se aplica si hay más de una variable predictora
    print("Matriz de Correlación entre Variables Predictoras:")
    correlation_matrix = X.corr()
    print(correlation_matrix)

    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Matriz de Correlación de Variables Predictoras')
    plt.show()

    print("\nFactor de Inflación de la Varianza (VIF):")
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(vif_data)

    print("\nInterpretación del VIF:")
    print("  - VIF = 1: No hay correlación.")
    print("  - VIF entre 1 y 5: Moderada correlación.")
    print("  - VIF > 5 (o > 10, según la literatura): Alta multicolinealidad, lo que puede ser problemático.")
else:
    print("Solo hay una variable predictora (X), por lo que la multicolinealidad no es un problema en este modelo.")
print("-" * 50)

## Supuesto 5: Independencia de los Residuos (Ausencia de Autocorrelación) 🔗
# Los residuos no deben estar correlacionados entre sí. Esto es crucial en series de tiempo.
# Se verifica con el estadístico de Durbin-Watson, que está incluido en el resumen de `statsmodels`.

print("\n--- Evaluación del Supuesto de Independencia de los Residuos ---")
# Extrae la estadistica de Durbiny-Watson
summary_string = str(results.summary())
durbin_watson_line = [line for line in summary_string.split('\n') if 'Durbin-Watson' in line][0]
durbin_watson_stat = float(durbin_watson_line.split()[-1])

print(f"Estadístico Durbin-Watson (del resumen del modelo): {durbin_watson_stat:.3f}")
print("Interpretación del Durbin-Watson:")
print("  - Valor **cercano a 2**: No hay autocorrelación.")
print("  - Valor **< 2**: Autocorrelación positiva (residuos adyacentes similares).")
print("  - Valor **> 2**: Autocorrelación negativa (residuos adyacentes opuestos).")
print("  - Un valor entre 1.5 y 2.5 generalmente se considera aceptable.")
print("-" * 50)

print("\n--- Conclusión de la Evaluación de Supuestos ---")
print("La evaluación de estos supuestos es crucial para confiar en las inferencias del modelo de regresión lineal.")
print("Si los supuestos no se cumplen, las estimaciones de los coeficientes y sus errores estándar pueden no ser válidas,")
print("afectando la significancia estadística y la capacidad de generalización del modelo.")
print("En caso de incumplimiento, se pueden considerar transformaciones de datos, la inclusión de otras variables,")
print("o el uso de modelos de regresión más avanzados (ej. regresión robusta, modelos generalizados lineales).")

In [ ]:
data = pd.read_excel("/content/sample_data/Data_Regresion.xlsx", sheet_name='data')
data.head()

##Mínimos Cuadrados Generalizados Factibles con GLSAR

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot, shapiro, jarque_bera, normaltest, anderson, kstest
from statsmodels.stats.diagnostic import het_breuschpagan, het_white,lilliefors
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import GLS, GLSAR



# Transformar 'Puntaje(y)' (as per your original code)
#data['Puntaje(y)'] = data['Puntaje(y)']**3

# --- 1. Definir variables dependiente (Y) e independiente (X) ---
X = data[['Horas(x)']] # X debe ser un DataFrame para sm.add_constant
y = data['Puntaje(y)']

# Añadir una constante a las características para el modelo statsmodels
X_const = sm.add_constant(X)

print("Datos Cargados:")
print(data)
print("-" * 50)

# --- 2. Construcción del Modelo de Regresión Lineal (OLS inicial para diagnóstico) ---
# Usamos statsmodels para tener un resumen detallado y acceso a los residuos para los supuestos.

model_ols = sm.OLS(y, X_const)
results_ols = model_ols.fit()
print("Resumen del Modelo de Regresión Lineal (OLS Inicial):")
print(results_ols.summary())
print("-" * 50)

# Obtener los residuos del modelo OLS
residuals_ols = results_ols.resid
fitted_values_ols = results_ols.fittedvalues

# --- 3. Evaluación de Supuestos de la Regresión Lineal (con OLS) ---

## Supuesto 1: Linealidad 📈
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.scatterplot(x=X['Horas(x)'], y=y)
plt.title('Linealidad: X vs. Y (OLS)')
plt.xlabel('X')
plt.ylabel('Y')

plt.subplot(1, 2, 2)
sns.scatterplot(x=fitted_values_ols, y=residuals_ols)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos vs. Valores Predichos (OLS)')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuos')
plt.tight_layout()
plt.show()

print("\n--- Evaluación del Supuesto de Linealidad (OLS) ---")
print("Observar el gráfico 'X vs. Y': si los puntos siguen una tendencia lineal, el supuesto se cumple.")
print("En el gráfico de 'Residuos vs. Valores Predichos', los residuos deben estar distribuidos aleatoriamente alrededor de cero, sin patrones evidentes (forma de embudo, curva, etc.).")
print("-" * 50)

## Supuesto 2: Normalidad de los Residuos 📊
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(residuals_ols, kde=True)
plt.title('Histograma de los Residuos (OLS)')
plt.xlabel('Residuos')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
probplot(residuals_ols, dist="norm", plot=plt)
plt.title('Q-Q Plot de los Residuos (OLS)')
plt.tight_layout()
plt.show()

print("\n--- Evaluación del Supuesto de Normalidad de los Residuos (OLS) ---")
print("Pruebas estadísticas:")

shapiro_test = shapiro(residuals_ols)
print(f"Shapiro-Wilk Test: Estadístico={shapiro_test.statistic:.3f}, p-value={shapiro_test.pvalue:.3f}")
if shapiro_test.pvalue > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

jarque_bera_test = jarque_bera(residuals_ols)
print(f"Jarque-Bera Test: Estadístico={jarque_bera_test.statistic:.3f}, p-value={jarque_bera_test.pvalue:.3f}")
if jarque_bera_test.pvalue > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

agostino_test = normaltest(residuals_ols)
print(f"\nAgostino-Pearson Test: Estadístico={agostino_test.statistic:.3f}, p-value={agostino_test.pvalue:.3f}")
if agostino_test.pvalue > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

anderson_test = anderson(residuals_ols, dist='norm')
print(f"\nAnderson-Darling Test: Estadístico={anderson_test.statistic:.3f}")
print("Valores críticos y niveles de significancia:")
for i in range(len(anderson_test.critical_values)):
    sl, cv = anderson_test.significance_level[i], anderson_test.critical_values[i]
    if anderson_test.statistic < cv:
        print(f"   -> Para nivel de significancia {sl}%: estadístico < {cv:.3f} (No rechazar normalidad)")
    else:
        print(f"   -> Para nivel de significancia {sl}%: estadístico >= {cv:.3f} (Rechazar normalidad)")

ks_test = kstest(residuals_ols, 'norm', args=(residuals_ols.mean(), residuals_ols.std()))
print(f"\nKolmogorov-Smirnov Test: Estadístico={ks_test.statistic:.3f}, p-value={ks_test.pvalue:.3f}")
if ks_test.pvalue > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

lillie_test = lilliefors(residuals_ols, dist='norm')
print(f"\nLilliefors Test: Estadístico={lillie_test[0]:.3f}, p-value={lillie_test[1]:.3f}")
if lillie_test[1] > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: los residuos parecen ser normales (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: los residuos no parecen ser normales (p <= 0.05).")

print("\nObservar el Q-Q Plot: los puntos deben seguir aproximadamente la línea recta. Las desviaciones indican no normalidad.")
print("-" * 50)

## Supuesto 3: Homocedasticidad (Varianza Constante de los Residuos) 📏
print("\n--- Evaluación del Supuesto de Homocedasticidad (OLS) ---")
print("Pruebas estadísticas:")

bp_test = het_breuschpagan(residuals_ols, X_const)
print(f"Breusch-Pagan Test: LM Statistic={bp_test[0]:.3f}, p-value={bp_test[1]:.3f}, F-statistic={bp_test[2]:.3f}, F-p-value={bp_test[3]:.3f}")
if bp_test[1] > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: hay homocedasticidad (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: hay heterocedasticidad (p <= 0.05).")

white_test = het_white(residuals_ols, X_const)
print(f"White Test: LM Statistic={white_test[0]:.3f}, p-value={white_test[1]:.3f}, F-statistic={white_test[2]:.3f}, F-p-value={white_test[3]:.3f}")
if white_test[1] > 0.05:
    print("   -> **No se puede rechazar la hipótesis nula**: hay homocedasticidad (p > 0.05).")
else:
    print("   -> **Se rechaza la hipótesis nula**: hay heterocedasticidad (p <= 0.05).")
print("-" * 50)

## Supuesto 4: Ausencia de Multicolinealidad 🔄
print("\n--- Evaluación del Supuesto de Ausencia de Multicolinealidad ---")
if X.ndim > 1 and X.shape[1] > 1: # Solo se aplica si hay más de una variable predictora
    print("Matriz de Correlación entre Variables Predictoras:")
    correlation_matrix = X.corr()
    print(correlation_matrix)

    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Matriz de Correlación de Variables Predictoras')
    plt.show()

    print("\nFactor de Inflación de la Varianza (VIF):")
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(vif_data)

    print("\nInterpretación del VIF:")
    print("   - VIF = 1: No hay correlación.")
    print("   - VIF entre 1 y 5: Moderada correlación.")
    print("   - VIF > 5 (o > 10, según la literatura): Alta multicolinealidad, lo que puede ser problemático.")
else:
    print("Solo hay una variable predictora (X), por lo que la multicolinealidad no es un problema en este modelo.")
print("-" * 50)

## Supuesto 5: Independencia de los Residuos (Ausencia de Autocorrelación) 🔗
print("\n--- Evaluación del Supuesto de Independencia de los Residuos (OLS) ---")
summary_string_ols = str(results_ols.summary())
durbin_watson_line_ols = [line for line in summary_string_ols.split('\n') if 'Durbin-Watson' in line][0]
durbin_watson_stat_ols = float(durbin_watson_line_ols.split()[-1])

print(f"Estadístico Durbin-Watson (del resumen del modelo OLS): {durbin_watson_stat_ols:.3f}")
print("Interpretación del Durbin-Watson:")
print("   - Valor **cercano a 2**: No hay autocorrelación.")
print("   - Valor **< 2**: Autocorrelación positiva (residuos adyacentes similares).")
print("   - Valor **> 2**: Autocorrelación negativa (residuos adyacentes opuestos).")
print("   - Un valor entre 1.5 y 2.5 generalmente se considera aceptable.")
print("-" * 50)

# --- Implementación de FGLS para Autocorrelación (usando GLSAR) ---
# GLSAR es una implementación de GLS para corregir la autocorrelación.
# Estima el parámetro de autocorrelación (rho) y luego realiza la transformación.

print("\n--- Implementación de FGLS para corregir Autocorrelación (usando GLSAR) ---")

# Para GLSAR, necesitamos un modelo GLSAR y el número de lags para el AR(p) proceso.
# Generalmente, se empieza con AR(1) (lags=1) si el Durbin-Watson indica autocorrelación positiva.
# Si el Durbin-Watson es bajo (mucho menor que 2), sugiere autocorrelación positiva de primer orden.

# Ajustar un modelo GLSAR con un proceso AR(1)
model_glsar = GLSAR(y, X_const, 1) # 1 indica AR(1)
results_glsar = model_glsar.iterative_fit()

print("\nResumen del Modelo de Regresión con FGLS (GLSAR - AR(1)):")
print(results_glsar.summary())
print("-" * 50)

# Obtener los residuos y valores predichos del modelo GLSAR
residuals_glsar = results_glsar.resid
fitted_values_glsar = results_glsar.fittedvalues

# --- Re-evaluación del Supuesto de Independencia de los Residuos (con FGLS) ---
print("\n--- Re-evaluación del Supuesto de Independencia de los Residuos (después de FGLS) ---")
summary_string_glsar = str(results_glsar.summary())
durbin_watson_line_glsar = [line for line in summary_string_glsar.split('\n') if 'Durbin-Watson' in line][0]
durbin_watson_stat_glsar = float(durbin_watson_line_glsar.split()[-1])

print(f"Estadístico Durbin-Watson (del resumen del modelo GLSAR): {durbin_watson_stat_glsar:.3f}")
print("Interpretación del Durbin-Watson:")
print("   - Valor **cercano a 2**: No hay autocorrelación.")
print("   - Valor **< 2**: Autocorrelación positiva (residuos adyacentes similares).")
print("   - Valor **> 2**: Autocorrelación negativa (residuos adyacentes opuestos).")
print("   - Un valor entre 1.5 y 2.5 generalmente se considera aceptable.")
print("\nDespués de aplicar FGLS, esperamos que el estadístico Durbin-Watson se acerque a 2.")
print("-" * 50)

# --- Visualización de Residuos después de FGLS ---
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(x=fitted_values_glsar, y=residuals_glsar)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos vs. Valores Predichos (GLSAR)')
plt.xlabel('Valores Predichos (GLSAR)')
plt.ylabel('Residuos (GLSAR)')

plt.subplot(1, 2, 2)
sns.histplot(residuals_glsar, kde=True)
plt.title('Histograma de Residuos (GLSAR)')
plt.xlabel('Residuos (GLSAR)')
plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()

print("\n--- Conclusión de la Evaluación de Supuestos ---")
print("La evaluación de estos supuestos es crucial para confiar en las inferencias del modelo de regresión lineal.")
print("Si los supuestos no se cumplen, las estimaciones de los coeficientes y sus errores estándar pueden no ser válidas,")
print("afectando la significancia estadística y la capacidad de generalización del modelo.")
print("En caso de incumplimiento, se pueden considerar transformaciones de datos, la inclusión de otras variables,")
print("o el uso de modelos de regresión más avanzados (ej. regresión robusta, modelos generalizados lineales).")



##Mínimos cuadrados ponderados (wls)



In [ ]:
# --- Implementación de Mínimos Cuadrados Ponderados (WLS) ---

print("\n--- Implementación de Mínimos Cuadrados Ponderados (WLS) ---")

# 1. Explicación de WLS
print("\n**Concepto de Mínimos Cuadrados Ponderados (WLS)**")
print("Mínimos Cuadrados Ponderados (WLS) es un método de regresión que se utiliza")
print("cuando el supuesto de homocedasticidad de los errores se viola (es decir, hay heterocedasticidad).")
print("La heterocedasticidad implica que la varianza de los errores no es constante")
print("a lo largo de los diferentes niveles de las variables predictoras.")
print("En WLS, se asignan pesos a cada observación. Las observaciones con menor varianza (errores más pequeños)")
print("reciben pesos mayores, y las observaciones con mayor varianza (errores más grandes)")
print("reciben pesos menores. Esto le da más 'importancia' en el ajuste del modelo a las observaciones")
print("que son más fiables (tienen menor varianza), corrigiendo así el problema de la heterocedasticidad")
print("y produciendo estimaciones de coeficientes más eficientes (menores errores estándar).")
print("Generalmente, los pesos son inversamente proporcionales a la varianza de los errores.")

# 2. Definir los pesos
# Una forma común de estimar los pesos es usar los residuos del modelo OLS inicial.
# Si la varianza de los errores es proporcional a alguna función de las variables predictoras
# o de los valores ajustados, podemos usar esa función para definir los pesos.
# Por ejemplo, si la varianza es proporcional a los valores ajustados al cuadrado (fitted_values_ols**2),
# los pesos serían inversamente proporcionales a fitted_values_ols**2.
# Para evitar dividir por cero o valores muy pequeños, a menudo se añade una pequeña constante.

# Estimación de los pesos: Inverso de la varianza estimada de los residuos OLS
# Una aproximación simple es usar el inverso de los residuos al cuadrado del modelo OLS.
# Sin embargo, esto puede ser problemático si los residuos son cercanos a cero.
# Una alternativa común es usar el inverso de los valores ajustados al cuadrado o alguna transformación de X.
# Aquí, usaremos el inverso de los valores ajustados del modelo OLS.
# Se recomienda añadir una pequeña constante para evitar la división por cero.
weights = 1.0 / (fitted_values_ols**2 + 1e-6) # Añadir una pequeña constante para estabilidad

print("\nPesos calculados (basados en el inverso de los valores ajustados del modelo OLS):")
print(weights.head())
print("-" * 50)

# 3. Ajustar el modelo WLS
# Usamos la clase WLS de statsmodels, pasando la variable dependiente, las predictoras con constante, y los pesos.
model_wls = sm.WLS(y, X_const, weights=weights)
results_wls = model_wls.fit()

# 4. Mostrar el resumen de los resultados del modelo WLS
print("\nResumen del Modelo de Regresión con WLS:")
print(results_wls.summary())
print("-" * 50)

# 5. Explicar cómo los pesos influyen en el ajuste y los resultados
print("\n**Influencia de los Pesos en el Modelo WLS**")
print("En el modelo WLS, las observaciones con pesos más altos (generalmente aquellas con menor varianza de error estimada)")
print("tienen una mayor influencia en la determinación de los coeficientes de regresión.")
print("Esto significa que el modelo se ajusta más estrechamente a los puntos de datos que se consideran más 'fiables' (menos ruidosos).")
print("La principal ventaja es que las estimaciones de los coeficientes son más eficientes (tienen menores errores estándar)")
print("en presencia de heterocedasticidad, en comparación con OLS.")
print("Comparando el resumen de WLS con el de OLS, se observará que los errores estándar de los coeficientes son diferentes,")
print("y potencialmente menores, lo que puede afectar la significancia estadística (valores p).")
print("El R-cuadrado en WLS se interpreta de manera diferente a OLS y a menudo no es directamente comparable.")
print("-" * 50)


# Obtener los residuos y valores predichos del modelo WLS
residuals_wls = results_wls.resid
fitted_values_wls = results_wls.fittedvalues

# 6. Incluir visualizaciones (residuos ponderados vs. valores ajustados)
# Para evaluar la homocedasticidad después de aplicar WLS, es útil graficar los residuos ponderados
# (residuos * sqrt(weights)) o los residuos sin ponderar frente a los valores ajustados.
# Idealmente, los residuos ponderados vs. valores ajustados deberían mostrar una dispersión más uniforme alrededor de cero.

# Calcular residuos ponderados
weighted_residuals_wls = residuals_wls * np.sqrt(weights)

plt.figure(figsize=(12, 6))

# Gráfico de Residuos WLS sin ponderar vs. Valores Predichos WLS
plt.subplot(1, 2, 1)
sns.scatterplot(x=fitted_values_wls, y=residuals_wls)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos (WLS) vs. Valores Predichos (WLS)')
plt.xlabel('Valores Predichos (WLS)')
plt.ylabel('Residuos (WLS)')

# Gráfico de Residuos Ponderados WLS vs. Valores Predichos WLS
plt.subplot(1, 2, 2)
sns.scatterplot(x=fitted_values_wls, y=weighted_residuals_wls)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos Ponderados (WLS) vs. Valores Predichos (WLS)')
plt.xlabel('Valores Predichos (WLS)')
plt.ylabel('Residuos Ponderados (WLS)')

plt.tight_layout()
plt.show()

print("\n--- Evaluación de Homocedasticidad (después de WLS) ---")
print("Observar el gráfico de 'Residuos Ponderados (WLS) vs. Valores Predichos (WLS)'.")
print("Esperamos que la dispersión de los puntos sea más uniforme alrededor de la línea cero")
print("en comparación con el gráfico de residuos vs. valores predichos del modelo OLS inicial.")
print("Esto indicaría que WLS ha ayudado a corregir la heterocedasticidad.")
print("-" * 50)


## Regresión Lineal Robusta
.


In [ ]:
# --- Implementación de Regresión Lineal Robusta ---

print("\n--- Implementación de Regresión Lineal Robusta ---")

# 1. Explicación de Regresión Lineal Robusta
print("\n**Concepto de Regresión Lineal Robusta**")
print("La Regresión Lineal Robusta es una alternativa a Mínimos Cuadrados Ordinarios (OLS)")
print("que es menos sensible a la presencia de valores atípicos (outliers) o influyentes")
print("en los datos. Mientras que OLS minimiza la suma de los errores al cuadrado,")
print("lo que hace que los valores atípicos tengan un impacto cuadrático grande en el ajuste,")
print("la regresión robusta utiliza diferentes funciones de pérdida o esquemas de ponderación")
print("para mitigar la influencia de estas observaciones extremas.")
print("Se utiliza típicamente cuando la inspección visual de los datos o el análisis de residuos")
print("del modelo OLS inicial sugiere la presencia de outliers que podrían distorsionar los resultados de OLS.")

# 2. Métodos de Regresión Robusta en statsmodels
print("\n**Métodos de Regresión Robusta en `statsmodels`**")
print("`statsmodels` implementa varios métodos de estimación robusta.")
print("Los más comunes son los M-estimators (Estimadores M).")
print("Los M-estimators generalizan la idea de OLS minimizando una función de pérdida (rho)")
print("de los residuos, en lugar del cuadrado de los residuos. Equivalente a esto,")
print("ponderan las observaciones de manera iterativa en función del tamaño de sus residuos;")
print("los residuos más grandes reciben un peso menor.")
print("Algunas funciones de ponderación (normas) comunes para M-estimators incluyen:")
print("  - Huber's T (HuberT): Pondera linealmente los residuos grandes.")
print("  - Tukey's Biweight (TukeyBiweight): Asigna peso cero a residuos que exceden un cierto umbral,")
print("    eliminando efectivamente su influencia.")
print("La clase `statsmodels.api.RLM` (Robust Linear Model) implementa M-estimators.")
print("Por defecto, `RLM` utiliza la norma de Huber (HuberT).")

# 3. Importar la clase RLM (ya importada en una celda anterior)
# from statsmodels.api import RLM # No es necesario importar de nuevo

# 4. Instanciar un modelo de regresión robusta
# Usamos la clase RLM de statsmodels. Podemos especificar una función de ponderación (norm).
# Por defecto, usa HuberT. Aquí usaremos HuberT explícitamente o dejaremos el valor por defecto.
# También podemos usar TukeyBiweight: sm.robust.norms.TukeyBiweight()

model_robust = sm.RLM(y, X_const, M=sm.robust.norms.HuberT())
# O simplemente: model_robust = sm.RLM(y, X_const) # Usa HuberT por defecto

# 5. Ajustar el modelo robusto
results_robust = model_robust.fit()

# 6. Mostrar el resumen de los resultados del modelo de regresión robusta
print("\nResumen del Modelo de Regresión Robusta (usando HuberT):")
print(results_robust.summary())
print("-" * 50)

In [ ]:
# 7. Obtener los residuos y valores predichos del modelo robusto
residuals_robust = results_robust.resid
fitted_values_robust = results_robust.fittedvalues

print("\nResiduos del modelo Robusto (HuberT):")
print(residuals_robust.head())
print("\nValores predichos del modelo Robusto (HuberT):")
print(fitted_values_robust.head())
print("-" * 50)

# 8. Visualizar los residuos del modelo robusto
plt.figure(figsize=(12, 6))

# Gráfico de Residuos Robusto vs. Valores Predichos Robusto
plt.subplot(1, 2, 1)
sns.scatterplot(x=fitted_values_robust, y=residuals_robust)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos (Robusto - HuberT) vs. Valores Predichos (Robusto)')
plt.xlabel('Valores Predichos (Robusto)')
plt.ylabel('Residuos (Robusto)')

# Gráfico de Residuos OLS vs. Valores Predichos OLS para comparación (usando variables de celdas anteriores)
plt.subplot(1, 2, 2)
sns.scatterplot(x=fitted_values_ols, y=residuals_ols)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos (OLS) vs. Valores Predichos (OLS) - Comparación')
plt.xlabel('Valores Predichos (OLS)')
plt.ylabel('Residuos (OLS)')

plt.tight_layout()
plt.show()

print("\n--- Evaluación Visual de Residuos (Robusto vs. OLS) ---")
print("Observar el gráfico de 'Residuos (Robusto - HuberT) vs. Valores Predichos (Robusto)'.")
print("La dispersión de los puntos puede ser diferente a la del gráfico OLS,")
print("especialmente si hay valores atípicos. Los puntos atípicos que tenían")
print("una gran desviación en OLS pueden tener una menor influencia o un patrón diferente")
print("en el modelo robusto debido a la reponderación.")
print("-" * 50)


# 9. Imprimir una breve interpretación de los resultados del modelo robusto
print("\n--- Interpretación de los Resultados del Modelo Robusto (HuberT) ---")
print("Comparando el resumen del Modelo Robusto con el del Modelo OLS inicial:")
print(f"- Coeficiente 'const' (Robusto): {results_robust.params['const']:.3f} (OLS: {results_ols.params['const']:.3f})")
print(f"- Coeficiente 'Horas(x)' (Robusto): {results_robust.params['Horas(x)']:.3f} (OLS: {results_ols.params['Horas(x)']:.3f})")
print(f"- Error Estándar 'const' (Robusto): {results_robust.bse['const']:.3f} (OLS: {results_ols.bse['const']:.3f})")
print(f"- Error Estándar 'Horas(x)' (Robusto): {results_robust.bse['Horas(x)']:.3f} (OLS: {results_ols.bse['Horas(x)']:.3f})")

print("\nObservaciones:")
print("Los coeficientes estimados por el modelo robusto pueden diferir de los de OLS,")
print("especialmente si hay valores atípicos que influyen en el ajuste de OLS.")
print("Los errores estándar del modelo robusto a menudo se calculan de manera diferente")
print("(por defecto, usa Cov Type H1 en statsmodels RLM, que es robusta a la heterocedasticidad)")
print("y pueden proporcionar inferencias más fiables en presencia de desviaciones de los supuestos de OLS.")
print("La regresión robusta, al ponderar menos los outliers, tiende a ajustar la 'tendencia central' de los datos,")
print("siendo menos arrastrada por puntos extremos.")
print("En este caso, el coeficiente de 'Horas(x)' es ligeramente diferente y el intercepto también,")
print("lo que sugiere que el modelo robusto ha ajustado el ajuste para ser menos influenciado por")
print("las observaciones con residuos grandes (los posibles outliers identificados en el análisis OLS anterior).")
print("-" * 50)

## Modelos Lineales Generalizados



In [ ]:
# --- Implementación de Modelos Lineales Generalizados (GLM) ---

print("\n--- Implementación de Modelos Lineales Generalizados (GLM) ---")

# 1. Explicación de GLM
print("\n**Concepto de Modelos Lineales Generalizados (GLM)**")
print("Los Modelos Lineales Generalizados (GLM) extienden el modelo de regresión lineal clásico (OLS)")
print("para permitir que la variable dependiente (respuesta) tenga una distribución de error")
print("distinta a la normal y para relacionar la media de la respuesta con una combinación")
print("lineal de las variables predictoras a través de una **función de enlace**.")
print("El modelo lineal clásico asume que:")
print("  - La variable dependiente es continua.")
print("  - Los errores son independientes y se distribuyen normalmente con media cero y varianza constante (homocedasticidad).")
print("  - La relación entre la media de la variable dependiente y las predictoras es lineal.")
print("\nGLM relaja estas suposiciones al especificar:")
print("  - Una **distribución de probabilidad** para la variable dependiente (ej. Binomial para datos binarios, Poisson para conteos, Gamma para datos positivos asimétricos).")
print("  - Una **función de enlace** que transforma la media esperada de la variable dependiente para que la relación con la combinación lineal de las predictoras sea lineal. G(E[y]) = Xβ.")
print("  - Una **función de varianza** que describe cómo la varianza de la respuesta depende de la media.")

# 2. Ejemplos comunes de GLM
print("\n**Ejemplos Comunes de GLM:**")
print("  - **Regresión Logística:** Para variables dependientes binarias (0/1). Utiliza la familia Binomial y la función de enlace logit.")
print("  - **Regresión de Poisson:** Para variables dependientes de conteo (números enteros no negativos). Utiliza la familia Poisson y la función de enlace log.")
print("  - **Regresión Gamma:** Para variables dependientes continuas y positivas con distribuciones asimétricas (ej. tiempos de espera, reclamaciones de seguros). Utiliza la familia Gamma y una función de enlace como la inversa o la log.")
print("  - **Regresión Lineal (como un caso especial de GLM):** Utiliza la familia Gaussiana (Normal) y la función de enlace identidad.")


# 3. Preparar datos para un ejemplo básico de GLM (Regresión de Poisson)
# El conjunto de datos actual ('data') es continuo y no es ideal para Poisson o Logística.
# Simularemos datos simples para una Regresión de Poisson.
# Supongamos que queremos modelar el número de eventos (un conteo) en función de una variable predictora.

np.random.seed(42) # para reproducibilidad
n_samples = 100
# Variable predictora (ej. nivel de exposición, tiempo)
X_sim = np.random.rand(n_samples) * 10
# Combinación lineal de predictoras (en este caso, solo una)
linear_predictor = 0.5 + 0.3 * X_sim
# Media esperada para una distribución de Poisson (lambda = exp(linear_predictor))
# La función de enlace log (log(lambda) = linear_predictor) es la canónica para Poisson.
mu_sim = np.exp(linear_predictor)
# Generar datos de conteo siguiendo una distribución de Poisson con la media calculada
y_sim = np.random.poisson(mu_sim)

# Crear un DataFrame para los datos simulados
data_glm = pd.DataFrame({'X_sim': X_sim, 'y_sim': y_sim})

# Añadir una constante a las características para el modelo statsmodels
X_sim_const = sm.add_constant(data_glm['X_sim'])
y_sim = data_glm['y_sim']

print("\nDatos Simulados para Regresión de Poisson:")
print(data_glm.head())
print("-" * 50)


In [ ]:
# 4. Instanciar un modelo GLM (Regresión de Poisson)
# Usamos la clase GLM de statsmodels, especificando la variable dependiente, las predictoras con constante,
# y la familia de distribución de errores (Poisson en este caso).

model_glm = sm.GLM(y_sim, X_sim_const, family=sm.families.Poisson())

# 5. Ajustar el modelo GLM
results_glm = model_glm.fit()

# 6. Mostrar el resumen de los resultados del modelo GLM
print("\nResumen del Modelo GLM (Regresión de Poisson):")
print(results_glm.summary())
print("-" * 50)

# 7. Explicar las diferencias clave en la interpretación de los resultados de un GLM
print("\n**Diferencias Clave en la Interpretación de Resultados (GLM vs. OLS)**")
print("Comparando el resumen del Modelo GLM (Poisson) con el del Modelo OLS:")
print("\n**Coeficientes:**")
print("  - En OLS, los coeficientes se interpretan directamente como el cambio esperado en la media de Y por un cambio de una unidad en X, manteniendo otras variables constantes.")
print("  - En GLM, los coeficientes se interpretan en la **escala de la función de enlace**. Para la Regresión de Poisson con enlace log, el coeficiente de una predictora")
print("    representa el cambio esperado en el **logaritmo de la media** de Y por un cambio de una unidad en X.")
print("    Esto significa que un cambio de una unidad en X multiplica la media esperada de Y por exp(coeficiente).")
print("    Por ejemplo, si el coeficiente es 0.3, un aumento de 1 en X multiplica la media esperada por exp(0.3) ≈ 1.35 (un aumento del 35%).")

print("\n**Bondad de Ajuste:**")
print("  - OLS utiliza R-cuadrado como métrica principal. Mide la proporción de la varianza en Y explicada por el modelo.")
print("  - GLM no usa R-cuadrado de la misma manera. Las métricas de bondad de ajuste para GLM a menudo se basan en la **devianza**, que es una generalización de la suma de cuadrados residuales.")
print("    La devianza residual mide la discrepancia entre el modelo ajustado y los datos saturados (un modelo que ajusta perfectamente cada observación).")
print("    Una devianza residual pequeña en relación con los grados de libertad sugiere un buen ajuste.")
print("    También se puede usar la devianza nula (deviance of the null model, con solo el intercepto) para calcular métricas análogas al R-cuadrado (pseudo R-squared),")
print("    como el Pseudo R-squared de McFadden, Cox-Snell, o Nagelkerke, aunque su interpretación es diferente a la de OLS.")
print("    El AIC (Criterio de Información de Akaike) y BIC (Criterio de Información Bayesiano) también se utilizan para comparar modelos GLM.")

print("\n**Supuestos:**")
print("  - OLS asume normalidad, homocedasticidad e independencia de los errores.")
print("  - GLM asume que la variable dependiente sigue la distribución de la familia especificada y que los errores son independientes.")
print("    No asume homocedasticidad (la varianza puede depender de la media, según la familia) ni normalidad de los errores en la escala original.")

print("\n**Errores Estándar y Significancia:**")
print("  - Los errores estándar y los valores p se interpretan de manera similar (para probar la significancia de los coeficientes),")
print("    pero se derivan de la teoría de máxima verosimilitud, que es la base de la estimación de GLM, en lugar de OLS.")

# 8. Concluir resaltando cuándo usar GLM
print("\n--- Cuándo Considerar el Uso de un GLM ---")
print("Debe considerar usar un GLM en lugar de OLS cuando:")
print("  - Su variable dependiente **no es continua** o **no tiene errores distribuidos normalmente**.")
print("    (ej. binaria, conteo, proporción, datos positivos asimétricos).")
print("  - La **varianza de los errores no es constante** y/o depende de la media de la respuesta.")
print("  - La relación entre la media de la respuesta y las predictoras **no es lineal** en la escala original,")
print("    pero puede ser lineal en la escala transformada por una función de enlace adecuada.")
print("GLM proporciona un marco flexible para modelar una variedad más amplia de tipos de datos de respuesta que OLS.")
print("-" * 50)

La Regresión por Mínimos Cuadrados Ponderados (WLS) aborda la heterocedasticidad asignando pesos a las observaciones, dando más importancia a aquellas con menor varianza de error. Un enfoque común es usar pesos inversamente proporcionales a los valores ajustados al cuadrado de un modelo OLS inicial. El resumen del modelo WLS muestra errores estándar ajustados en comparación con OLS, y los gráficos de residuos ponderados pueden indicar si la heterocedasticidad se ha mitigado.

Los Mínimos Cuadrados Generalizados (GLS) son un método más general que OLS y WLS, capaz de manejar tanto la heterocedasticidad como la autocorrelación. Su implementación en statsmodels requiere especificar la matriz de covarianza de error (Ω). Si Ω es diagonal con entradas inversamente proporcionales a los pesos de WLS, los resultados de GLS serán numéricamente idénticos a WLS. En la práctica, Ω es a menudo desconocida y necesita ser estimada (GLS Factible - FGLS).

La Regresión Lineal Robusta es menos sensible a los valores atípicos que OLS. Los estimadores M, como Huber's T o Tukey's Biweight, son métodos comunes que re-ponderan iterativamente las observaciones basándose en sus residuos, reduciendo la influencia de los residuos grandes. El resumen del modelo robusto y los gráficos de residuos pueden mostrar diferencias en las estimaciones de los coeficientes y los errores estándar de los errores en comparación con OLS, lo que refleja el impacto reducido de los valores atípicos.

Los Modelos Lineales Generalizados (GLM) extienden la regresión lineal a variables dependientes que no son continuas o normalmente distribuidas (por ejemplo, binarias, de recuento). Los GLM requieren especificar una distribución de probabilidad para la respuesta (por ejemplo, Binomial, Poisson) y una función de enlace para relacionar la media de la respuesta con el predictor lineal. La interpretación de los coeficientes se realiza en la escala de la función de enlace, no en la escala de respuesta original, y la bondad del ajuste se evalúa utilizando métricas como la deviance y el pseudo R-cuadrado en lugar del R-cuadrado de OLS.

